In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 62 kB/s 
     |████████████████████████████████| 198 kB 53.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=bf68d36fbcd2b16337b471251faa51e850c5e10388577fff537591a8136cb1de
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySparkDemo").getOrCreate()

In [ ]:
df = spark.createDataFrame([('Jack', 42),('Jill', 37),('John', 35),('Jane', 48),],['name','age'])

In [ ]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [ ]:
df.show()

+----+---+
|name|age|
+----+---+
|Jack| 42|
|Jill| 37|
|John| 35|
|Jane| 48|
+----+---+



In [ ]:
over40 = df.filter(df.age>40)
over40.show()

+----+---+
|name|age|
+----+---+
|Jack| 42|
|Jane| 48|
+----+---+



In [ ]:
rows = over40.count()
print(rows)

2


In [ ]:
#csv

from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

df = spark.read.option('header', True).csv('policy.csv')

df.printSchema()

root
 |-- policy: string (nullable = true)
 |-- make: string (nullable = true)
 |-- vehicle_age: string (nullable = true)
 |-- sum_insured: string (nullable = true)
 |-- inception_date: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)
 |-- premium: string (nullable = true)



In [ ]:
df.show(5)

+-------+------+-----------+-----------+--------------+----------+--------+-------+
| policy|  make|vehicle_age|sum_insured|inception_date|start_date|end_date|premium|
+-------+------+-----------+-----------+--------------+----------+--------+-------+
|CAR0001|TOYOTA|          1|      15000|      20180101|  20180101|20181231|   1000|
|CAR0001|TOYOTA|          2|      13500|      20180101|  20190101|20191231|    900|
|CAR0001|TOYOTA|          3|      12000|      20180101|  20200101|20201231|    800|
|CAR0002|SUBARU|          2|      14000|      20200210|  20200210|20210209|    950|
|CAR0003|  FORD|          6|      10000|      20180315|  20180315|20190314|    700|
+-------+------+-----------+-----------+--------------+----------+--------+-------+
only showing top 5 rows



In [ ]:
df2 = spark.read.option("header", True).csv("policy.csv") \
      .withColumn("sum_insured", col("sum_insured").cast(IntegerType())) \
      .withColumn("vehicle_age", col("vehicle_age").cast(IntegerType())) \
      .withColumn("premium", col("premium").cast(IntegerType()))

df2.show(5)
df2.printSchema()

+-------+------+-----------+-----------+--------------+----------+--------+-------+
| policy|  make|vehicle_age|sum_insured|inception_date|start_date|end_date|premium|
+-------+------+-----------+-----------+--------------+----------+--------+-------+
|CAR0001|TOYOTA|          1|      15000|      20180101|  20180101|20181231|   1000|
|CAR0001|TOYOTA|          2|      13500|      20180101|  20190101|20191231|    900|
|CAR0001|TOYOTA|          3|      12000|      20180101|  20200101|20201231|    800|
|CAR0002|SUBARU|          2|      14000|      20200210|  20200210|20210209|    950|
|CAR0003|  FORD|          6|      10000|      20180315|  20180315|20190314|    700|
+-------+------+-----------+-----------+--------------+----------+--------+-------+
only showing top 5 rows

root
 |-- policy: string (nullable = true)
 |-- make: string (nullable = true)
 |-- vehicle_age: integer (nullable = true)
 |-- sum_insured: integer (nullable = true)
 |-- inception_date: string (nullable = true)
 |-- s

In [ ]:
df2 = spark.read.option("header", True).csv("flights.csv") \
      .withColumn("mon", col("mon").cast(IntegerType())) \
      .withColumn("dom", col("dom").cast(IntegerType())) \
      .withColumn("dow", col("dow").cast(IntegerType())) \
      .withColumn("mile", col("mile").cast(IntegerType())) \
      .withColumn("duration", col("duration").cast(IntegerType()))

df2.show(5)
df2.printSchema()

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65|   NA|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows

root
 |-- mon: integer (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- org: string (nullable = true)
 |-- mile: integer (nullable = true)
 |-- depart: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- delay: string (nullable = true)



In [ ]:
#data transformation

from pyspark.sql import SparkSession
from pyspark.sql.functions import col,when,lit,concat,to_date,rank
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("PySparkTutorial").getOrCreate()

policyDF = spark.read.option("header", True).csv("policy.csv") \
      .withColumn("sum_insured", col("sum_insured").cast(IntegerType())) \
      .withColumn("vehicle_age", col("vehicle_age").cast(IntegerType())) \
      .withColumn("premium", col("premium").cast(IntegerType()))

claimsDF = spark.read.option("header", True).csv("claims.csv") \
      .withColumn("cost", col("cost").cast(IntegerType())) \

In [ ]:
policyDF.printSchema()
claimsDF.printSchema()

root
 |-- policy: string (nullable = true)
 |-- make: string (nullable = true)
 |-- vehicle_age: integer (nullable = true)
 |-- sum_insured: integer (nullable = true)
 |-- inception_date: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)
 |-- premium: integer (nullable = true)

root
 |-- policy: string (nullable = true)
 |-- incident_date: string (nullable = true)
 |-- cost: integer (nullable = true)



In [ ]:
#new column

policyDF = policyDF.withColumn("status", when(policyDF.start_date == policyDF.inception_date,
                                              "New Business").otherwise("Renewal"))

policyDF.show(4)

+-------+------+-----------+-----------+--------------+----------+--------+-------+------------+
| policy|  make|vehicle_age|sum_insured|inception_date|start_date|end_date|premium|      status|
+-------+------+-----------+-----------+--------------+----------+--------+-------+------------+
|CAR0001|TOYOTA|          1|      15000|      20180101|  20180101|20181231|   1000|New Business|
|CAR0001|TOYOTA|          2|      13500|      20180101|  20190101|20191231|    900|     Renewal|
|CAR0001|TOYOTA|          3|      12000|      20180101|  20200101|20201231|    800|     Renewal|
|CAR0002|SUBARU|          2|      14000|      20200210|  20200210|20210209|    950|New Business|
+-------+------+-----------+-----------+--------------+----------+--------+-------+------------+
only showing top 4 rows



In [ ]:
def fix_dates(df):
  for column in df.columns:
    if column.endswith("_date") and dict(df.dtypes)[column] == 'string':
      print("NOTE: Fixing date column '{}'.".format(column))
      df= df.withColumn(column,to_date(df[column],"yyyyMMdd"))
  return df

policyDF = fix_dates(policyDF)
claimsDF = fix_dates(claimsDF)

NOTE: Fixing date column 'incident_date'.


In [ ]:
#joining

combinedDF = policyDF.select(["policy","make","vehicle_age","sum_insured",
                              "start_date","end_date","premium"]) \
                              .join(claimsDF,(policyDF.policy == claimsDF.policy) &
                                    (policyDF.start_date <= claimsDF.incident_date) &
                                    (policyDF.end_date >= claimsDF.incident_date))
                              
combinedDF.show()

+-------+------+-----------+-----------+----------+----------+-------+-------+-------------+-----+
| policy|  make|vehicle_age|sum_insured|start_date|  end_date|premium| policy|incident_date| cost|
+-------+------+-----------+-----------+----------+----------+-------+-------+-------------+-----+
|CAR0001|TOYOTA|          3|      12000|2020-01-01|2020-12-31|    800|CAR0001|   2020-06-05| 5000|
|CAR0004| MAZDA|          5|      10000|2020-04-02|2021-04-01|    700|CAR0004|   2020-06-10| 3000|
|CAR0007|   BMW|          4|      24000|2020-07-13|2021-07-12|   1600|CAR0007|   2020-09-10|24000|
|CAR0009| TESLA|          1|      72000|2019-09-17|2020-09-16|   4800|CAR0009|   2020-02-10|15000|
+-------+------+-----------+-----------+----------+----------+-------+-------+-------------+-----+



In [ ]:
combinedDF = policyDF.select(["policy","make","vehicle_age","sum_insured",
                              "start_date","end_date","premium"]).join(claimsDF,(policyDF.policy == claimsDF.policy))
                              
combinedDF.show()

+-------+------+-----------+-----------+----------+----------+-------+-------+-------------+-----+
| policy|  make|vehicle_age|sum_insured|start_date|  end_date|premium| policy|incident_date| cost|
+-------+------+-----------+-----------+----------+----------+-------+-------+-------------+-----+
|CAR0001|TOYOTA|          1|      15000|2018-01-01|2018-12-31|   1000|CAR0001|   2020-06-05| 5000|
|CAR0001|TOYOTA|          2|      13500|2019-01-01|2019-12-31|    900|CAR0001|   2020-06-05| 5000|
|CAR0001|TOYOTA|          3|      12000|2020-01-01|2020-12-31|    800|CAR0001|   2020-06-05| 5000|
|CAR0004| MAZDA|          4|      11000|2019-04-02|2020-04-01|    750|CAR0004|   2020-06-10| 3000|
|CAR0004| MAZDA|          5|      10000|2020-04-02|2021-04-01|    700|CAR0004|   2020-06-10| 3000|
|CAR0007|   BMW|          4|      24000|2020-07-13|2021-07-12|   1600|CAR0007|   2020-09-10|24000|
|CAR0009| TESLA|          1|      72000|2019-09-17|2020-09-16|   4800|CAR0009|   2020-02-10|15000|
|CAR0009| 

In [ ]:
policyDF = policyDF.withColumn("policy_term",rank().over(Window.partitionBy("policy").orderBy("start_date")))\
.withColumn("total_premium", F.sum("premium").over(Window.partitionBy("policy")
.orderBy("start_date")))\
.withColumn("policy_id",concat(policyDF.policy, lit("~"),col("policy_term")))

policyDF.select(["policy","start_date","policy_term","premium","total_premium","policy_id"]).show()

+-------+----------+-----------+-------+-------------+---------+
| policy|start_date|policy_term|premium|total_premium|policy_id|
+-------+----------+-----------+-------+-------------+---------+
|CAR0002|2020-02-10|          1|    950|          950|CAR0002~1|
|CAR0001|2018-01-01|          1|   1000|         1000|CAR0001~1|
|CAR0001|2019-01-01|          2|    900|         1900|CAR0001~2|
|CAR0001|2020-01-01|          3|    800|         2700|CAR0001~3|
|CAR0006|2020-06-18|          1|    300|          300|CAR0006~1|
|CAR0007|2020-07-13|          1|   1600|         1600|CAR0007~1|
|CAR0008|2020-08-11|          1|   1800|         1800|CAR0008~1|
|CAR0004|2019-04-02|          1|    750|          750|CAR0004~1|
|CAR0004|2020-04-02|          2|    700|         1450|CAR0004~2|
|CAR0009|2019-09-17|          1|   4800|         4800|CAR0009~1|
|CAR0009|2020-09-17|          2|   3500|         8300|CAR0009~2|
|CAR0010|2020-10-15|          1|    300|          300|CAR0010~1|
|CAR0005|2020-05-16|     

In [ ]:
#summarisation

summary = policyDF.groupBy("Status").sum("sum_insured")\
.withColumnRenamed("sum(sum_insured)","total_insured")

summary.show()

+------------+-------------+
|      Status|total_insured|
+------------+-------------+
|     Renewal|       107500|
|New Business|       199000|
+------------+-------------+



In [ ]:
policyDF.crosstab("status", "make").show()

+------------+----+---+----+------+-------+-----+------+------+-----+------+
| status_make|AUDI|BMW|FORD|HOLDEN|HYUNDAI|MAZDA|SUBARU|SUZUKI|TESLA|TOYOTA|
+------------+----+---+----+------+-------+-----+------+------+-----+------+
|New Business|   1|  1|   1|     1|      1|    2|     1|     1|    1|     1|
|     Renewal|   0|  0|   2|     0|      0|    0|     0|     0|    1|     2|
+------------+----+---+----+------+-------+-----+------+------+-----+------+



In [ ]:
#csv

from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

df = spark.read.option('header', True).csv('walmart_stock.csv')


df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [ ]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [ ]:
df.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import format_number
 
summary = df.describe()
summary.select(summary['summary'],
                  format_number(summary['Open'].cast('float'), 2).alias('Open'),
                  format_number(summary['High'].cast('float'), 2).alias('High'),
                  format_number(summary['Low'].cast('float'), 2).alias('Low'),
                  format_number(summary['Close'].cast('float'), 2).alias('Close'),
                  format_number(summary['Volume'].cast('int'),0).alias('Volume')
                 ).show()

df_hv = df.withColumn('HV Ratio', df['High']/df['Volume']).select(['HV Ratio'])
df_hv.show()

df.orderBy(df['High'].desc()).select(['Date']).head(1)[0]['Date']

from pyspark.sql.functions import mean
 
df.select(mean('Close')).show()

from pyspark.sql.functions import min, max
 
df.select(max('Volume'),min('Volume')).show()

df.filter(df['Close'] < 60).count()

+-------+--------+--------+--------+--------+----------+
|summary|    Open|    High|     Low|   Close|    Volume|
+-------+--------+--------+--------+--------+----------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|     1,258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8,222,093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4,519,780|
|    min|   56.39|   57.06|   56.30|   56.42|10,010,500|
|    max|   90.80|   90.97|   89.25|   90.47| 9,994,400|
+-------+--------+--------+--------+--------+----------+

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.

81

In [ ]:
from pyspark.sql.functions import mean
 
df.select(mean('Close')).show()


+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [ ]:
from pyspark.sql.functions import min, max
 
df.select(max('Volume'),min('Volume')).show()


+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|    9994400|   10010500|
+-----------+-----------+



In [ ]:
df.filter(df['Close'] < 60).count()

81

In [ ]:
df.filter('High > 80').count()  *  100/df.count()

8.426073131955485

In [ ]:
df3.write.mode('overwrite').parquet("./data/flights.parquet")

NameError: ignored

In [ ]:
#sql

df = spark.sql(""" SELECT policy
, make, CASE
WHEN inception_date = start_date THEN 'New Business'
ELSE 'Renewal'
END AS status

FROM parquet.'./data/policy.parquet'
ORDER BY policy,start_date"""
)
df.show()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 26))



ParseException: ignored

In [ ]:
#parallelize

spark = SparkSession.builder.appName("EX1").getOrCreate()

rdd = spark.sparkContext.parallelize([1,2,3,4,5])

rddCollect = rdd.collect()

print("Number of partitions: "+str(rdd.getNumPartitions()))
print("Actions:First element: "+str(rdd.first()))

print(rddCollect)

emptyRDD = spark.sparkContext.emptyRDD()
emptyRDD2 = rdd= spark.sparkContext.parallelize([])

print(""+str(emptyRDD2.isEmpty()))

Number of partitions: 2
Actions:First element: 1
[1, 2, 3, 4, 5]
True


In [ ]:
from pyspark.sql.types import IntegerType

flights = flights.withColumn("air_time",flights["air_time"].cast(IntegerType()))
flights = flights.withColumn("dep_delay",flights["dep_delay"].cast(IntegerType()))
flights.printSchema()

AnalysisException: ignored

In [ ]:
flights = spark.read.option('header', True).csv('flights.csv')

In [ ]:
long_flights1 = flights.filter('distance > 1000')

long_flights2 = flights.filter(flights.distance > 1000)

long_flights1.show(5)
long_flights2.show(5)

AnalysisException: ignored

In [ ]:
from pyspark.sql.functions import length

data = data.withColumn('length', length(data['text']))
data.show()

NameError: ignored